# 🛫 External Stuff

## 🚗 DMVs

In [8]:
set nocount on; 
select * from sys.database_scoped_credentials;
select * from sys.external_data_sources;
select * from sys.external_file_formats;

## ⚓ Infrastructure

In [7]:
if exists (select 1 from sys.external_file_formats where name ='parquet_file')      drop external file format parquet_ff;
if exists (select 1 from sys.external_file_formats where name ='csv2')              drop external file format csv2_ff;
if exists (select 1 from sys.external_file_formats where name ='csv')               drop external file format csv_ff;
if exists (select 1 from sys.external_data_sources where name ='MyAzureStorage')    drop external data source hiramblob_ds;
if exists (select 1 from sys.database_scoped_credentials where name ='AzureStorageCredential')  drop database scoped credential hiramblob_cred;

In [9]:
create database scoped credential hiramblob_cred
with 
     identity = 'sqlva4onabrnvvfrkw',
     secret = 'mykey' --mykey
;

create external data source hiramblob_ds
with (    
    location = 'wasbs://hiramblob@sqlva4onabrnvvfrkw.blob.core.windows.net/',
    credential = hiramblob_cred,
    type = HADOOP
);

Commands completed successfully.

Total execution time: 00:00:00.198

## 📝 Files

In [10]:
if exists (select 1 from sys.external_file_formats where name = 'parquet_ff') drop external file format parquet_ff;
create external file format parquet_ff
with (
    format_type = parquet,  
    data_compression = 'org.apache.hadoop.io.compress.SnappyCodec'  
);

if exists (select 1 from sys.external_file_formats where name = 'csv2_ff') drop external file format csv2_ff;
create external file format csv2_ff
with (
    format_type = delimitedtext,
    format_options(
        field_terminator = ',',
        string_delimiter = '"',
        first_row = 2, 
        use_type_default = true
    )
);

if exists (select 1 from sys.external_file_formats where name = 'csv_ff') drop external file format csv_ff;
create external file format csv_ff
with (
    format_type = delimitedtext,
    format_options(
        field_terminator = ',',
        string_delimiter = '"',
        use_type_default = true
    )
);

Commands completed successfully.

Total execution time: 00:00:00.276

## 🎁 CETAS - Create External Table As Select (outbound)

In [0]:
if exists (select * from sys.external_tables where name ='demoparquet')
begin 
    drop external table demoparquet;
end;
if exists (select * from sys.external_tables where name ='extBand')
begin 
    drop external table extBand;
end;
if exists (select * from sys.external_tables where name ='extBand2')
begin 
    drop external table extBand2;
end;

In [0]:
create external table demoparquet (
    BandId  int,
    Band    nvarchar(50),
    Album   nvarchar(50)
)
with (   
    location = 'demoparquet',  
    data_source = hiramblob_ds,  
    file_format = parquet_ff  
)
as 
    select * from dbo.Band;
go

create external table extBand
with (   
    location = 'extBand',  
    data_source = hiramblob_ds,  
    file_format = csv_ff  
)
as 
    select * from dbo.Band;
go

create external table extBand2
with (   
    location = 'extBand2',  
    data_source = hiramblob_ds,  
    file_format = csv2_ff  
)
as 
select * from dbo.Band;

In [0]:
exec sp_spaceused Band;
select count(*) from extBand;
select count(*) from extBand2;

## 🔎 CTAS - Create Table As Select (inboud)
##### https://github.com/microsoft/sql-server-samples/tree/master/samples/features/sql-big-data-cluster/data-virtualization/storage-pool

In [0]:
if exists (select * from sys.external_tables where name ='ctasBand')
begin 
    drop external table ctasBand;
end;

In [0]:
create external table ctasBand (
    BandId  int,
    Band    nvarchar(50),
    Album   nvarchar(50)
)
with (
    datasource = hiramblob_ds,
    location = '/extBand',
    file_format = csv2_ff
);

In [0]:
exec sp_spaceused ctasBand;
select count(*) from ctasBand;